In [1]:
# imports a library 'pandas', names it as 'pd'

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV


# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [2]:
read_in_file = pd.read_csv('C:/Users/smnio/Desktop/Projects/local_only/appt1.csv')#, nrows=1000000)

In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [4]:
read_in_file = read_in_file.dropna()
df= read_in_file.copy()
df.head()

,﻿app_month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,eth_refused,eth_hispanic,lang_Eng,lang_Chi,lang_Spa,lang_Other,male,female,income,depression
0,2,16,5,1,0,0,0,0,1,0.750000,...,0,0,1,0,0,0,0,1,83622,0
1,5,9,5,1,0,0,1,0,0,0.769230,...,0,0,1,0,0,0,0,1,83622,0
2,5,14,2,1,0,0,1,0,0,1.000000,...,0,0,1,0,0,0,0,1,83622,0
3,10,10,5,1,0,0,1,0,0,0.857142,...,0,0,1,0,0,0,0,1,83622,0
4,10,11,3,1,0,0,1,0,0,0.833333,...,0,0,1,0,0,0,0,1,83622,0


In [5]:
cols = ['month']
for i in range(1,len(df.columns)):
    cols.append(df.columns[i])
df.columns=cols

df = pd.concat([df, pd.get_dummies(df['month'], prefix='month')], axis=1)

In [6]:
df = pd.concat([df, pd.get_dummies(df['weekday'], prefix='wkday')], axis=1)

In [7]:
df = pd.concat([df, pd.get_dummies(df['appt_begintime'], prefix='hour')], axis=1)

In [8]:
df.income = df.income.round(-4)
df = pd.concat([df, pd.get_dummies(df['income'], prefix='inc')], axis=1)

In [9]:
#df['income'] = df.income.map(lambda x: df['income'].median() if (x > 0))

In [10]:
#break out ages into discrete groups

df['age_65_and_over'] = 0
df['age_under_5'] = 0
df['age_5_17'] = 0
df['age_18_29'] = 0
df['age_30_44'] = 0
df['age_45_64'] = 0


df['age_65_and_over'] = df.age.map(lambda x: 1 if (x > 64) else 0)
df['age_under_5'] = df.age.map(lambda x: 1 if (x < 5) else 0)
df['age_5_17'] = df.age.map(lambda x: 1 if (x > 4 and x <18 ) else 0)
df['age_18_29'] = df.age.map(lambda x: 1 if (x > 17 and x <30 ) else 0)
df['age_30_44'] = df.age.map(lambda x: 1 if (x > 29 and x <45 ) else 0)
df['age_45_64'] = df.age.map(lambda x: 1 if (x > 44 and x <65 ) else 0)

df.head()

,month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,inc_190000,inc_200000,inc_210000,inc_220000,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
0,2,16,5,1,0,0,0,0,1,0.750000,...,0,0,0,0,1,0,0,0,0,0
1,5,9,5,1,0,0,1,0,0,0.769230,...,0,0,0,0,1,0,0,0,0,0
2,5,14,2,1,0,0,1,0,0,1.000000,...,0,0,0,0,0,0,0,0,0,1
3,10,10,5,1,0,0,1,0,0,0.857142,...,0,0,0,0,0,0,0,0,0,1
4,10,11,3,1,0,0,1,0,0,0.833333,...,0,0,0,0,0,0,0,0,0,1


In [11]:
columnNames=df.columns

In [12]:
from sklearn import preprocessing

xi = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
xi_scaled = min_max_scaler.fit_transform(xi)
df = pd.DataFrame(xi_scaled)


df.columns=columnNames



In [13]:
df.corr()

,month,appt_begintime,weekday,kept,rescheduled,cancelled,last_kept,last_rescheduled,last_cancelled,kept_perc,...,inc_190000,inc_200000,inc_210000,inc_220000,age_65_and_over,age_under_5,age_5_17,age_18_29,age_30_44,age_45_64
month,1.000000,-0.000094,-0.024945,0.028282,-0.022948,-0.015123,0.024512,-0.021563,-0.011676,0.020970,...,-0.000655,0.000236,0.000080,0.000525,0.003701,0.005986,0.008141,0.005766,-0.010956,-0.003996
appt_begintime,-0.000094,1.000000,-0.064270,-0.026138,0.010195,0.022441,-0.041433,0.018605,0.033933,-0.055069,...,-0.001589,0.002008,-0.000750,0.001779,-0.071025,-0.002215,0.105701,0.048736,0.011070,-0.027221
weekday,-0.024945,-0.064270,1.000000,-0.001329,-0.000461,0.001894,-0.010256,0.005075,0.008026,-0.010393,...,-0.002295,0.001522,-0.000990,-0.000764,-0.012127,0.002621,0.024131,0.007358,-0.007516,0.000030
kept,0.028282,-0.026138,-0.001329,1.000000,-0.523660,-0.755884,0.118493,-0.072906,-0.081372,0.215198,...,-0.000478,-0.000643,0.001702,0.000761,0.047475,0.032662,0.006178,-0.029242,-0.017449,-0.023245
rescheduled,-0.022948,0.010195,-0.000461,-0.523660,1.000000,-0.161936,-0.066130,0.120695,-0.018239,-0.132621,...,0.001103,-0.000891,-0.000891,-0.000399,-0.022332,-0.011291,0.000402,-0.008898,0.003859,0.027022
cancelled,-0.015123,0.022441,0.001894,-0.755884,-0.161936,1.000000,-0.086432,-0.008306,0.108271,-0.147348,...,-0.000294,0.001430,-0.001287,-0.000575,-0.037829,-0.029156,-0.007464,0.040709,0.017246,0.006158
last_kept,0.024512,-0.041433,-0.010256,0.118493,-0.066130,-0.086432,1.000000,-0.543100,-0.744143,0.530135,...,0.001270,-0.000715,0.001657,0.000741,0.051711,0.030548,0.004789,-0.027235,-0.018168,-0.026546
last_rescheduled,-0.021563,0.018605,0.005075,-0.072906,0.120695,-0.008306,-0.543100,1.000000,-0.156771,-0.284678,...,0.000140,0.000853,-0.000900,-0.000403,-0.025329,-0.011773,-0.000521,-0.007989,0.003645,0.029973
last_cancelled,-0.011676,0.033933,0.008026,-0.081372,-0.018239,0.108271,-0.744143,-0.156771,1.000000,-0.397073,...,-0.001605,0.000162,-0.001233,-0.000552,-0.040672,-0.026565,-0.005219,0.038391,0.018469,0.007378
kept_perc,0.020970,-0.055069,-0.010393,0.215198,-0.132621,-0.147348,0.530135,-0.284678,-0.397073,1.000000,...,0.002160,0.001874,0.002234,0.001322,0.098484,0.056673,0.009174,-0.047530,-0.038290,-0.049897


In [14]:
#df.corr()['kept'].order(ascending=False)

In [15]:
#k=corrVals[corrVals.ix[:1]>.001]
#print(k)

In [16]:
corr_series = df.corr()['kept'].order(ascending=False)
#pd.set_option('display.height', 500)
#pd.set_option('display.max_rows', 500)

print(corr_series)

kept                1.000000
kept_perc           0.215198
last_kept           0.118493
income              0.066574
lang_Eng            0.056879
age_65_and_over     0.047475
age_under_5         0.032662
month               0.028282
age                 0.027164
inc_100000          0.025166
male                0.025060
inc_120000          0.024854
inc_90000           0.022595
eth_refused         0.019693
inc_110000          0.018390
hour_2              0.017670
hour_3              0.017135
hour_5              0.017013
hour_1              0.016535
hour_4              0.016252
hour_6              0.015140
hour_12             0.014974
month_10            0.013382
inc_130000          0.011904
inc_150000          0.011723
eth_not_hispanic    0.009616
month_11            0.009430
inc_160000          0.008543
inc_140000          0.008537
inc_170000          0.008257
                      ...   
hour_19            -0.006327
hour_15            -0.006455
lang_Spa           -0.008641
hour_23       

In [17]:
selected_columns = corr_series[((corr_series > .02 ) | (corr_series < -.02))].index

selected_columns = selected_columns[((selected_columns != 'kept' ) & (selected_columns != 'cancelled') & (selected_columns != 'rescheduled'))]

print(selected_columns)

Index([u'kept_perc', u'last_kept', u'income', u'lang_Eng', u'age_65_and_over',
       u'age_under_5', u'month', u'age', u'inc_100000', u'male', u'inc_120000',
       u'inc_90000', u'month_1', u'age_45_64', u'female', u'appt_begintime',
       u'age_18_29', u'inc_50000', u'inc_30000', u'month_2', u'lang_Chi',
       u'last_rescheduled', u'last_cancelled', u'resch_perc', u'can_perc'],
      dtype='object')


In [18]:

X = df.ix[:, selected_columns] 

y = df.ix[:, df.columns == 'kept']




In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [20]:


#np.all(np.isfinite(mat))

In [21]:
#supervised classification
from sklearn.linear_model import LogisticRegression

lr=LogisticRegression()

#lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

C:\Users\smnio\Anaconda\lib\site-packages\sklearn\utils\validation.py:449: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.71681766891556942

In [22]:
#0.71724745080587426

In [23]:
read_in_file = read_in_file.dropna()



y_pred = lr.predict(X)



data = pd.DataFrame({"Actual":read_in_file.kept, "Predicted":y_pred.tolist()})



In [24]:
#get calculated coeffcicients
lr.coef_

array([[ 0.59383017,  0.19380058,  0.7385156 , -0.02536959,  0.17079182,
         0.39949307, -0.01206626, -0.195883  ,  0.0034381 ,  0.11044716,
        -0.01653647,  0.00429322, -0.19357421,  0.01892426,  0.06068731,
        -0.24064183, -0.10303737, -0.12162198, -0.06881833, -0.27846698,
        -0.20697647,  0.20054103,  0.18543152, -1.36074716, -1.08993446]])

In [25]:
import csv

data_top=data.head(10000)

data_top.to_csv("predictions.csv")

